# Context
> Utilities for pulling context

In [ ]:
#|default_exp context

In [ ]:
#|export
import os, re, requests
from fastcore.script import call_parse
from concurrent.futures import ThreadPoolExecutor, as_completed
from urllib.parse import urlparse, urljoin
import pandoc
from requests.exceptions import RequestException
import xml.etree.ElementTree as ET
from tqdm import tqdm

In [ ]:
#|export
def _sanitize_filename(name):
    invalid_chars = '<>:"/\\|?*'
    for char in invalid_chars:
        name = name.replace(char, '_')
    # Strip existing file extension if any when markdown is expected
    name = os.path.splitext(name)[0]
    return name

def _html_to_markdown_pandoc(html_content, strip_html=False):
    p = pandoc.read(source=html_content, format='html')
    md = pandoc.write(p, format='markdown_strict')
    if strip_html: md = re.sub(r'<[^>]*>', '', md)
    return md

def _fetch_and_save(url:str, output_dir:str, to_markdown:bool):
    try:
        response = requests.get(url, timeout=10)
        response.raise_for_status()
        content = response.content  # Assuming binary content to handle non-text files as well

        parsed_url = urlparse(url)
        filename = _sanitize_filename(parsed_url.path.strip('/').replace('/', '_') or 'index')
        # Determine the file extension based on to_markdown flag
        file_extension = '.md' if to_markdown else '.html'
        filename += file_extension

        filepath = os.path.join(output_dir, filename)
        os.makedirs(os.path.dirname(filepath), exist_ok=True)

        with open(filepath, 'wb' if not to_markdown else 'w', encoding=None if not to_markdown else 'utf-8') as f:
            if to_markdown:
                # Convert HTML to markdown if needed
                content = _html_to_markdown_pandoc(content.decode('utf-8'))
            f.write(content)
        return url, None
    except RequestException as e:
        return url, f"HTTP error for URL '{url}': {e}"
    except Exception as e:
        return url, f"Failed to fetch or save URL '{url}': {e}"

@call_parse
def fetch_site_from_sitemap(url:str, # The site that you want to fetch documents from 
                            output_dir:str, # The output directory to write documents to, 
                            to_markdown:bool=True # Write output as markdown (recommended)
                           ) :
    """
    Fetch content from the specified URL and save it to the given directory.

    Saves the content as a markdown file if `to_markdown` is True. Otherwise, it saves the content as an HTML file. 
    """
    
    if not url.endswith('/sitemap.xml'):
        url = urljoin(url, '/sitemap.xml')

    try:
        sitemap_response = requests.get(url, timeout=10)
        sitemap_response.raise_for_status()
        sitemap_xml = sitemap_response.text
    except RequestException as e:
        print(f"HTTP error when accessing sitemap: {e}")
        return
    except Exception as e:
        print(f"Unexpected error when downloading sitemap: {e}")
        return

    try:
        root = ET.fromstring(sitemap_xml)
        namespaces = {'ns': root.tag.split('}')[0].strip('{')}
        urls = [url.text for url in root.findall('ns:url/ns:loc', namespaces)]
    except ET.ParseError as e:
        print(f"Error parsing the XML sitemap: {e}")
        return
    except Exception as e:
        print(f"Unexpected error when parsing sitemap: {e}")
        return

    os.makedirs(output_dir, exist_ok=True)

    tasks = []
    with ThreadPoolExecutor() as executor:
        for url in urls:
            tasks.append(executor.submit(_fetch_and_save, url, output_dir, to_markdown))

        for future in tqdm(as_completed(tasks), total=len(tasks), unit='file'):
            url, error = future.result()
            if error:
                print(error)

In [ ]:
fetch_site_from_sitemap('https://quarto.org', 
                        output_dir='quarto_files/')

100%|████████████████████████████████████████████████████████████| 291/291 [00:29<00:00,  9.95file/s]


In [ ]:
!ls quarto_files/ | head -n5

about.md
bug-reports.md
docs_authoring_appendices.md
docs_authoring_article-layout.md
docs_authoring_callouts.md


In [ ]:
!cat quarto_files/about.md | tail -n50


        # configure for your platform (./configure.sh or ./configure.cmd for windows)
        $ cd quarto-cli
        $ ./configure.sh

        # checkout a new branch
        $ git checkout -b feature/newthing

2.  For significant changes (e.g more than small bug fixes), ensure that
    you have signed the
    [individual](https://posit.co/wp-content/uploads/2023/04/2023-03-13_TC_Indiv_contrib_agreement.pdf)
    or
    [corporate](https://posit.co/wp-content/uploads/2023/04/2023-03-13_TC_Corp_contrib_agreement.pdf)
    contributor agreement as appropriate. You can send the signed copy
    to <jj@rstudio.com>.

3.  Submit the [pull
    request](https://help.github.com/articles/using-pull-requests). It
    is ok to submit as draft in your are still working on it but would
    like some feedback from us. It always good to share in the open that
    you are working on it.

We’ll try to be as responsive as possible in reviewing and accepting
pull requests.

Proudly supported by [<img
src=